In [17]:
# rvarandas nan value code (modified)
import numpy as np
from os import listdir

In [18]:
path = "../training/"
paths = [path + "p0" + str(10000+i)[1:] + ".psv" for i in range(1, 5001)]

In [19]:
# Modified data load using ../ paths
data = np.array([np.loadtxt(open(path + file), delimiter='|', skiprows=1) for file in paths])
keys = open(paths[0]).readline().rstrip().split('|')

In [20]:
# Nan value correction
def delete_columns_nan(data,keys):
    df = {}
    for i, column in enumerate(data.T):
        if not np.isnan(column).all():
            df[keys[i]] = column
    return df


def replace_nan_by_value(data, value=None):
    for j, patient in enumerate(data):
        for key in patient.keys():
            if value == 'normal':
                for i in range(len(patient[key])):
                    p = patient[key].copy()[~np.isnan(patient[key])]
                    if np.isnan(patient[key][i]):
                        if value == 'mean':
                            data[j][key][i] = np.mean(p)
                        if value == 'normal':
                            data[j][key][i] = np.random.normal(np.mean(p), np.std(p))
            else:
                patient[key] = np.nan_to_num(patient[key])
    return data

In [21]:
# Remove NaN-ONLY columns?
data_aux = []
for patient in data:
    data_aux.append(delete_columns_nan(patient, keys))

# Count non-EMPTY entries throughout patients
df = {}
for key in keys:
    df[key] = 0

for patient in data_aux:
    for key in patient.keys():
        df[key] += 1
  
for key,val in df.items():
    print (key,":", val)
    
    
data_new = replace_nan_by_value(data_aux, None)


HR : 4999
O2Sat : 5000
Temp : 4990
SBP : 4998
MAP : 4983
DBP : 4996
Resp : 4984
EtCO2 : 784
BaseExcess : 131
HCO3 : 90
FiO2 : 1439
pH : 1423
PaCO2 : 1424
SaO2 : 1265
AST : 2115
BUN : 4605
Alkalinephos : 2118
Calcium : 4621
Chloride : 394
Creatinine : 4606
Bilirubin_direct : 355
Glucose : 4723
Lactate : 1171
Magnesium : 4109
Phosphate : 2873
Potassium : 4649
Bilirubin_total : 2119
TroponinI : 1693
Hct : 4521
Hgb : 4526
PTT : 1080
WBC : 4512
Fibrinogen : 481
Platelets : 4514
Age : 5000
Gender : 5000
Unit1 : 3509
Unit2 : 3509
HospAdmTime : 5000
ICULOS : 5000
SepsisLabel : 5000


## pat_feat Patients that have feature X

In [22]:
# Get the list of patients that have feature X
pat_feat = {}
for i in range(len(keys)):
    L = []
    for j in range(5000):
        if keys[i] in data_new[j]:
#            print(i, j) 
            L.append(j)              
    pat_feat[keys[i]] = L

In [23]:
len(pat_feat["Temp"])

4990

## Intersection

In [24]:
# Set intersection between lists
def Intersection(lst1, lst2): 
    return set(lst1).intersection(lst2) 

In [25]:
# Get the shared-feature list of patients
def get_sharedfeat_patients(npatients = 5000, features = None, initial_list = None):
    intersec = -1
    if features == None:
        print("Error: features must be passed")
    else:
        if initial_list == None:
            intersec = range(1,npatients + 1)
        else: 
            intersec = initial_list

        for i in range(len(features)):
            intersec = Intersection(intersec, pat_feat[features[i]])
        return intersec

In [26]:
intersec_patients = get_sharedfeat_patients(5000, keys)

In [27]:
intersec_patients

{130, 2845, 3206, 4452}

## Find which features are shared

In [28]:
# Get features that given patients share
def get_shared_features(npatients = 5000, features = None, initial_list = None):
    # print(npatients, len(features), len(initial_list)
    myfeat = -1
    if features == None:
        print("Error: features must be passed")
    else:
        if initial_list == None:
            initial_list = list(range(npatients))
    
        myfeat = []
        for i in range(len(features)):
            if len(Intersection(initial_list, pat_feat[features[i]])) == len(initial_list):
                myfeat.append(features[i])
    return myfeat

In [29]:
# Features shared by everyone
my_feat = get_shared_features(5000, keys)
my_feat

['O2Sat', 'Age', 'Gender', 'HospAdmTime', 'ICULOS', 'SepsisLabel']

In [30]:
# Features shared by those who present SaO2
patient_list = pat_feat["SaO2"]
my_feat = get_shared_features(5000, keys, patient_list)
my_feat

['HR',
 'O2Sat',
 'SBP',
 'DBP',
 'SaO2',
 'Age',
 'Gender',
 'HospAdmTime',
 'ICULOS',
 'SepsisLabel']

# Find Sepsis Features 

In [32]:
data_aux = []
sepsis_keys = {}
df = {}

for key in keys:
    df[key] = 0
    sepsis_keys[key] = 0

for patient in data:
    data_aux.append(delete_columns_nan(patient, keys))
    if np.any(data_aux[-1]['SepsisLabel']) == 1:
        for key in data_aux[-1].keys():
            sepsis_keys[key] += 1

keys_s = []
for key in sepsis_keys.keys():
    if sepsis_keys[key] == np.max(list(sepsis_keys.values())):
        keys_s.append(key)
print(keys_s)

['HR', 'O2Sat', 'SBP', 'MAP', 'DBP', 'Age', 'Gender', 'HospAdmTime', 'ICULOS', 'SepsisLabel']
